# Path Optimization (KOMO)



Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path += ['../build', '../../../build', '../../lib']
import numpy as np
import libry as ry

C = ry.Config()
D = C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}



Let's add some objects

In [2]:
C.addObject(name='item1', parent='sink1', shape=ry.ST.ssBox, pos=[-.1, -.1, .52], size=[.1, .1, .25, .02], color=[1., 0., 0.])
C.addObject('item2', 'sink1', ry.ST.ssBox, [.1, .1, .25, .02], [1., 1., 0.], [.1, .1, .52])
C.addObject('tray', 'stove1', ry.ST.ssBox, [.2, .2, .05, .02], [0., 1., 0.], [.0, .0, .42])

compute a collision free path to touch object item1 with pr2L:

In [3]:
X0 = C.getFrameState()

obj1 = "item1";
arm = "pr2L";

komo = C.komo_path(1.,20, 10., True);

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.distance, [arm, obj1], ry.OT.eq, [1e2]);
komo.addObjective([.9,1.], ry.FS.positionDiff, ["endeffWorkspace", obj1], ry.OT.sos, [1e0]);
komo.addObjective(times=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);

komo.optimize()
komo.getReport()

** KOMO::run solver:sparse collisions:1 x-dim:504 T:20 k:2 phases:1 stepsPerPhase:20 tau:0.5  #timeSlices:22 #totalDOFs:504 #frames:3982
** optimization time:0.185974 (kin:0.015012 coll:0.076105 feat:0.055451 newton: 0.011259) setJointStateCount:76
   sos:16.7659 ineq:6.11781 eq:0.234833


{'F_qItself/2-#46': {'order': 2.0, 'type': 'sos', 'sos': 14.175057128562791},
 'F_AccumulatedCollisions/0-#181': {'order': 0.0,
  'type': 'eq',
  'eq': 0.00546248503171232},
 'F_qLimits/0-#181': {'order': 0.0, 'type': 'ineq', 'ineq': 6.117809029880054},
 'F_PairCollision/0-pr2L-item1': {'order': 0.0,
  'type': 'eq',
  'eq': 0.0104931816480313},
 'F_PositionDiff/0-endeffWorkspace-item1': {'order': 0.0,
  'type': 'sos',
  'sos': 2.5908831094842206},
 'F_qItself/1-#58': {'order': 1.0, 'type': 'eq', 'eq': 0.21887745956131638},
 'sos': 16.765940238047012,
 'ineq': 6.117809029880054,
 'eq': 0.23483312624106,
 'f': 0.0}

In [6]:
komo.view(False, "my solution")

0

In [9]:
komo.view_play(False, .1)

0

Best way to read out: grab a configuration into K and analyze it here:

In [8]:
C.setFrameState( komo.getFrameState(19) )
C.getJointState()

array([-1.65041089e-01,  1.17961582e+00,  1.07655667e+00,  1.00018506e-01,
       -6.53769462e-03, -3.97708685e-03,  6.38756319e-01,  1.20875787e+00,
        4.10034810e-01,  1.23179531e-01,  4.33873545e-01, -1.33912969e+00,
        9.32187255e-01, -8.12933638e-01, -2.05854677e+00, -3.71741832e-04,
        6.96270668e-05, -9.40137260e-02, -5.04446960e-01, -9.84603917e-05,
        1.91666731e-05,  9.95645402e-02,  9.76504053e-02,  7.65864660e-03,
        5.93171958e-03])